In [1]:
import scipy.stats as st
import numpy as np
import math


class ConfidenceIntervals():
    
    def __init__(self, data, mean):
        self.data = data
        self.mean = mean
        
    def ci(self, alpha):
        return self.mean - 2 * np.std(self.data, ddof=1), self.mean + 2 * np.std(self.data, ddof=1)
    

class ZScoreCI(ConfidenceIntervals):
    
    def __init__(self, data, mean, sigma):
        ConfidenceIntervals.__init__(self, data, mean)
        self.sigma = sigma
    
    def ci(self, alpha):
        z_alpha_half = st.norm.ppf(1 - alpha / 2)
        se = z_alpha_half * self.sigma / math.sqrt(len(self.data))
        return self.mean - se, self.mean + se
    

class TScoreCI(ConfidenceIntervals):
    
    def ci(self, alpha):
        s_n = np.std(self.data, ddof=1)
        t_alpha_half = st.t.ppf(1 - alpha / 2, len(self.data) - 1)
        se = t_alpha_half * s_n / math.sqrt(len(self.data))
        return self.mean - se, self.mean + se
    
    
class BootstrapCI(ConfidenceIntervals):
    
    def __init__(self, data, mean, sampler, size):
        ConfidenceIntervals.__init__(self, data, mean)
        self.sampler = sampler
        self.size = size
        
    def ci(self, alpha):
        samples = self.sampler.sample(self.data, self.mean, size=self.size)
        delta = np.mean(samples, axis=0) - self.mean
        return self.mean - np.quantile(delta, 1 - alpha / 2), self.mean - np.quantile(delta, alpha / 2)

    
class Sampler:
    
    def __init__(self, sampling_func):
        self.sampling_func = sampling_func
    
    def sample(self, data, mean, size):
        return self.sampling_func(data, (len(data), size))


data = st.norm.rvs(size=20)
zscore = ZScoreCI(data, np.mean(data), 1)
print(zscore.ci(0.05))

tscore = TScoreCI(data, np.mean(data))
print(tscore.ci(0.05))

bootci = BootstrapCI(data, np.mean(data), Sampler(np.random.choice), 1000)
print(bootci.ci(0.05))

(-0.3475668941560758, 0.5289556464205057)
(-0.2938043442753138, 0.4751930965397438)
(-0.23905160308558632, 0.4491412165328954)
